In [1]:
device_id = 2
device = 'cuda:' + str(device_id)
#device = 'cpu'

import os
import re
import math
import time
import pickle
import json
import torch, torch.nn as nn
import numpy as np

from multiprocessing import pool


from pkg.util import read_problemsheet, write_problemsheet, write_answersheet
from pkg.parse import *
from pkg.words import *
from problems import *

import sentencepiece as spm

from pkg.vocab import Vocab, CharVocab, SPVocab

Problems = [P1_1_1, P1_1_2, P1_1_3, P1_1_4, P1_1_5, P1_1_6, P1_1_7, P1_1_8, P1_1_9, P1_1_10, P1_1_11, P1_1_12, 
            P1_2_1, P1_2_2, P1_3_1, P1_4_1, 
            P2_1_1, P2_2_2, P2_3_1, 
            P3_1_1, P3_2_1, P3_2_2, P3_3_1, 
            P4_1_1, P4_2_1, P4_2_2, P4_3_1, 
            P5_1_1, P5_2_1, P5_3_1,
            P6_1_1, P6_3_1, P6_4_1,
            P7_1_1, P7_1_2, P7_3_1,
            P8_1_1, P8_2_1, P8_3_1, 
            P9_1_1, P9_2_1, P9_2_2, P9_3_1, P9_3_2]

In [2]:
dir_trained = 'trained'
dir_question_classifier = os.path.join(dir_trained, 'question_classifier')
path_cfg = os.path.join(dir_question_classifier, 'cfg.pkl')
path_model = os.path.join(dir_question_classifier, 'trained.pth')

dir_tokenization = os.path.join(dir_trained, 'tokenization')
path_vocab_model = os.path.join(dir_tokenization, 'prob_512.model')

path_questions = 'sample.json'

problems = read_problemsheet(path_questions, normalize=True)
vocab = SPVocab(path_vocab_model)

In [3]:
with open(path_cfg, 'rb') as f:
    cfg = pickle.load(f)[0]
QC = cfg.create_object()    
state_dict = torch.load(path_model, map_location='cpu')['models'][0]
QC.load_state_dict(state_dict)


<All keys matched successfully>

In [4]:
class Solver:
    def __init__(self, qc, vocab, problems, parser, device='cpu'):
        self.qc = qc.eval().to(device)
        self.vocab = vocab
        self.problems = problems
        self.parser = parser
        self.device = device
    
    def normalize_text(self, text):
        text = text.replace('?', '')
        text = text.replace('  ', ' ')
        text = text.strip()
        return text   
   
    @torch.no_grad()
    def sort_problem_matching(self, question, normalize=True):
        if normalize:
            question = self.normalize_text(question)
        idx = self.vocab.encode_as_ids(question)
        idx = torch.tensor(idx, dtype=torch.int64).unsqueeze(0)
        logit = self.qc(idx.to(self.device)).squeeze(0).cpu().numpy()
        order = (-logit).argsort().tolist()
        return order
    
    def try_solve(self, question, porder, normalize=True):
        if normalize:
            question = self.normalize_text(question)
        for pid in porder:
            problem = self.problems[pid]
            print(pid, problem.__name__)
            objects, numbers, variables, formulas, equations, lists = self.parser(question)
            try:
                result = problem.try_solve(numbers, variables, formulas, equations, lists, question)
                if result is None:
                    continue
                else:
                    answer, equation = result
                    return answer, equation
            except Exception as ex:
                print(ex)
                continue
        return '', ''
    
    def solve(self, question, normalize=True):
        porder = self.sort_problem_matching(question, normalize)
        answer, equation = self.try_solve(question, porder, normalize)
        return answer, equation


In [5]:
solver = Solver(QC, vocab, Problems, parse)

In [6]:
questions = read_problemsheet('sample.json')


In [7]:
start = time.time()
solutions = {}
for k, q in questions.items():
    print('---------------', k, '-------------')
    print('question:', q)
    answer, equation = solver.solve(q)
    solutions[k] = {"answer":answer, "equation":equation}
    print('answer:', answer)
    print('equation:', equation)
print(time.time() - start)
print('done')

--------------- 1 -------------
question: 상자 안에 9개의 공이 있습니다. 석진이가 5개의 공을 상자 안에 더 넣었습니다. 상자 안에 있는 공은 모두 몇 개입니까?
12 P1_2_1
answer: 14
equation: print(9 + 5)
--------------- 2 -------------
question: 1부터 200까지의 홀수의 합을 구하시오.
0 P1_1_1
answer: 10000
equation: sum = 0
for i in range(1, 200 + 1):
    if i % 2 == 1:
        sum += i
print(sum)
--------------- 3 -------------
question: 한 상자에는 감이 100개씩 들어있습니다. 6개의 상자 안에 있는 감은 모두 몇 개일까요?
14 P1_3_1
answer: 600
equation: x = 100
y = 6
print(x * y)
--------------- 4 -------------
question: 지민, 정국, 태형이의 수학 점수는 각각 94점, 82점, 88점입니다. 이 셋을 제외한 학급의 수학 점수 평균은 78점입니다. 지민이네 학급 인원수가 30명일 때, 학급 수학 평균 점수는 몇 점입니까?
15 P1_4_1
answer: 79
equation: N = 30
n = 3
scores = [94, 82, 88]
mean = 78
total = (N - n) * mean + sum(scores)
new_mean = total / 30
if new_mean / 1 == new_mean // 1:
    print(int(new_mean))
else:
    print('%.2f'%new_mean)
--------------- 5 -------------
question: 20명의 학생들이 한 줄로 줄을 섰습니다. 윤기의 앞에 11명의 학생들이 서 있습니다. 윤기의 뒤에 서 있는 학생은 몇 명입니까?
16 P2_1_1
ans

In [8]:
write_answersheet(solutions, 'answersheet.json')

In [9]:
problems

{'1': '상자 안에 9개의 공이 있습니다 석진이가 5개의 공을 상자 안에 더 넣었습니다 상자 안에 있는 공은 모두 몇 개입니까',
 '2': '1부터 200까지의 홀수의 합을 구하시오',
 '3': '한 상자에는 감이 100개씩 들어있습니다 6개의 상자 안에 있는 감은 모두 몇 개일까요',
 '4': '지민, 정국, 태형이의 수학 점수는 각각 94점, 82점, 88점입니다 이 셋을 제외한 학급의 수학 점수 평균은 78점입니다 지민이네 학급 인원수가 30명일 때, 학급 수학 평균 점수는 몇 점입니까',
 '5': '20명의 학생들이 한 줄로 줄을 섰습니다 윤기의 앞에 11명의 학생들이 서 있습니다 윤기의 뒤에 서 있는 학생은 몇 명입니까',
 '6': '달리기 시합에서 남준이는 2등을 했고, 윤기는 4등을 했습니다 호석이는 윤기보다 잘했지만 남준이보다는 못했습니다 호석이의 등수는 몇 등입니까',
 '7': '키가 작은 사람부터 순서대로 9명이 한 줄로 서 있습니다 호석이가 앞에서부터 5번째에 서 있습니다 키가 큰 사람부터 순서대로 다시 줄을 서면 호석이는 앞에서부터 몇 번째에 서게 됩니까',
 '8': '사과 7개를 서로 다른 2마리의 원숭이에게 나누어 주려고 합니다 원숭이는 적어도 사과 1개는 받습니다 사과를 나누어 주는 방법은 모두 몇 가지입니까',
 '9': '4개의 숫자 7, 2, 5, 9를 한 번씩만 사용하여 네 자리 수를 만들려고 합니다 만들 수 있는 네 자리 수는 모두 몇 개입니까',
 '10': '사과, 복숭아, 배, 참외 중에서 2가지의 과일을 골라서 사는 경우는 모두 몇 가지입니까',
 '11': '4개의 수 53, 98, 69, 84가 있습니다 그 중에서 가장 큰 수와 가장 작은 수의 차는 얼마입니까',
 '12': '어떤 소수의 소수점을 오른쪽으로 한 자리 옮기면 원래보다 2 7만큼 커집니다 원래의 소수를 구하시오',
 '13': '3개의 수 2, 3, 9로 나누어 떨어질 수 있는 세 자리 수는 모두 몇 개 있습니까',
 '14': '두

In [10]:
parse('두 자리 수의 덧셈식 A4+2B=69에서 A에 해당하는 숫자를 쓰시오')

SyntaxError: invalid syntax (<string>, line 1)

In [11]:
re_natural = '(?: *[0-9]+ *)'
re_float = '(?: *[0-9]+\.[0-9]+ *)'
re_fraction = '(?: *[0-9]+/[0-9]+ *)'
re_number = '(?:' + re_fraction + '|' + re_float + '|' + re_natural + ')'
re_variable = '(?: *[A-Z] *)'
re_alpha_numeric = '(?: *[0-9A-Z]+ *)'
re_formula_1 = '(?:' + '(?:' + re_alpha_numeric + '\+)+' + re_alpha_numeric + ')'
re_formula_0 = '(?:' + '(?:' + re_alpha_numeric + '\+)*' + re_alpha_numeric + ')'
re_equation = '(?:' + re_formula_0 + '=' + re_formula_0 + ')'

In [12]:
re.search(re_list['person'], '정국이는 4와 4를 모았다')


In [13]:
re_list['person']

'(?:(?:(?: *(?:(?:남준)|(?:석진)|(?:윤기)|(?:호석)|(?:지민)|(?:태형)|(?:정국)|(?:민영)|(?:유정)|(?:은지)|(?:유나)) *),)+(?: *(?:(?:남준)|(?:석진)|(?:윤기)|(?:호석)|(?:지민)|(?:태형)|(?:정국)|(?:민영)|(?:유정)|(?:은지)|(?:유나)) *))'